In [39]:
import os
from bs4 import BeautifulSoup, NavigableString, Tag 

from nltk.util import ngrams
from nltk import word_tokenize

import json
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as plt
import seaborn as sns

import collections
import csv

from datetime import datetime
from pytz import timezone

from pathlib2 import Path

print "imported"

imported


In [3]:
# IMPORTS SPECIFIC TO TOPIC MODELING

import langid
import nltk
import re
import time
from collections import defaultdict
from configparser import ConfigParser
from gensim import corpora, models, similarities
from nltk.tokenize import RegexpTokenizer
from string import digits
import pyLDAvis.gensim
print "imported"

imported


In [ ]:
# https://github.com/alexperrier/datatalks/blob/master/twitter/twitter_lda.py
# https://radimrehurek.com/gensim/wiki.html
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
# https://miningthedetails.com/blog/python/lda/GensimLDA/
# https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/lda_training_tips.ipynb

#### GENSIM

In [10]:
# INITIALIZE PARAMETERS

num_topics = 10
passes = 100
iterations = 400
eta = 'auto'
alpha = 'auto' #0.001
eval_every = None

# LOAD FILENAMES
# filepath = Path('data/dicts/')
# files = [filename.stem for filename in filepath.iterdir() if filename.is_file()]
# ['bnn_00_full','bnn_01_uni',...'wsj_06_lemmas','wsj_07_freq']

# TEMP B/C PROGRAM DID NOT RUN ALL MODELS
filepath = Path('data/dicts/')
files = [filename.stem for filename in filepath.iterdir() if (filename.is_file() and ('bnn' not in filename.name))]

# CREATE LDA MODELS
for filename in files:
    
    #INPUT PATHS 
    dict_path = 'data/dicts/'
    corpus_path = 'data/corpora/'

    dict_filename   = dict_path + filename + '.dict'
    corpus_filename = corpus_path + filename + '.mm'
    
    # NOT ENTIRELY CLEAR WHY I'VE CHOSEN TO USE A DICT HERE WHEN PARAMS ARE ABOVE...AH WELL...
    lda_params      = {'num_topics': num_topics, 'passes': passes, 'iterations': iterations, 
                       'eta':eta, 'alpha': alpha, 'eval_every': eval_every}

    # LOAD CORPUS AND DICTIONARY
    corpus = corpora.MmCorpus(corpus_filename)
    dictionary = corpora.Dictionary.load(dict_filename)
    
    # RUN LDA
    print("Running LDA for: %s with params %s" % (filename, lda_params.values()))
    lda = models.LdaModel(corpus, id2word=dictionary,
                          num_topics=lda_params['num_topics'], passes=lda_params['passes'],
                          iterations=lda_params['iterations'], eta=lda_params['eta'], 
                          alpha=lda_params['alpha'], eval_every=lda_params['eval_every'])
    
    # OUTPUT PATH
    lda_path = 'data/lda/gensim/'
    lda_filename    = lda_path + filename + '.lda'
    
    # SAVE MODEL
    lda.save(lda_filename)
    
    print("lda saved in %s \n" % lda_filename)
    
# Running LDA for: bnn_00_full with {'eval_every': None, 'passes': 100, 'num_topics': 10, 'eta': 'auto', 'iterations': 400, 'alpha': 'auto'}
# lda saved in data/lda/gensim/bnn_00_full.lda 

#### MALLET

#### NOTE: Not using at this time due to questions about removal of stop words. It would appear as though the convert_input function has this flag set to true.  This function is the first thing that runs inside of the train function below.

See: https://github.com/RaRe-Technologies/gensim/blob/8b810918d59781116794a6679999afdc76b857ef/gensim/models/wrappers/ldamallet.py#L246

In [6]:
# PART 16 https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

mallet_path = 'data/mallet-2.0.8/bin/mallet'

files = [filename.split('.')[0] for filename in os.listdir('data/dicts/') if not filename.startswith('.')]
files = files[1:] # REMOVE 'ALL' FOLDER
# ['cnn_level_zero', 'cnn_level_one',... 'month_d_level_five','month_d_level_six']# print files

for filename in files:
    
    dict_path = 'data/dicts/'
    corpus_path = 'data/corpora/'
    lda_path = 'data/lda/mallet/'

    dict_filename   = dict_path + filename + '.dict'
    corpus_filename = corpus_path + filename + '.mm'
    lda_filename    = lda_path + filename + '.lda'
    
    corpus = corpora.MmCorpus(corpus_filename)
    dictionary = corpora.Dictionary.load(dict_filename)
    
    lda = models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=dictionary)
    lda.save(lda_filename)
    
# bnn_level_one = lda.load('data/lda/bnn_level_one.lda')
# bnn_level_one.show_topics()